In [47]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import sys
import heapq
import seaborn as sns
from pylab import rcParams

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 12, 5

In [48]:
path = 'context.txt'
text = open(path,'r', encoding='UTF-8').read().lower()
print('corpus length:', len(text))

corpus length: 399147


In [49]:
chars = sorted(list(set(text)))
char_to_idx = dict((c,i) for (i,c) in enumerate(chars))
idx_to_char = dict((i,c) for (i,c) in enumerate(chars))
print('There are %d unique characters.' % len(chars))

There are 57 unique characters.


In [53]:
SEQUENCE_LENGTH = 40
step = 3
sentences = [] # training examples
next_chars = [] # prediction target
for i in range(0, len(text) - SEQUENCE_LENGTH, step):
    sentences.append(text[i:i+SEQUENCE_LENGTH])
    next_chars.append(text[i+SEQUENCE_LENGTH])
print('Numbers of training sequence:', len(sentences))

Numbers of training sequence: 133036


In [54]:
# One hot encoding
X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i,sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_idx[char]] = 1
    y[i,char_to_idx[next_chars[i]]] = 1

In [55]:
sentences[1535], next_chars[1535], X[1535][0], y[1535]

('er if we at last grow distrustful, lose ',
 'p',
 array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False]),
 array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
         True, False, False, False, False, False, False, False, False,
        False, False, False]))

In [57]:
X.shape, y.shape

((133036, 40, 57), (133036, 57))

In [65]:
# Define model
model = Sequential()
model.add(LSTM(128, input_shape=(SEQUENCE_LENGTH,len(chars)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer=RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [66]:
history = model.fit(X, y, validation_split=0.2, batch_size=128, epochs=20, shuffle=True).history

Epoch 1/20
832/832 [==============================] - 23s 24ms/step - loss: 2.5149 - accuracy: 0.2885 - val_loss: 2.0319 - val_accuracy: 0.4274
Epoch 2/20
832/832 [==============================] - 19s 23ms/step - loss: 1.7138 - accuracy: 0.4897 - val_loss: 1.9212 - val_accuracy: 0.4685
Epoch 3/20
832/832 [==============================] - 19s 22ms/step - loss: 1.5406 - accuracy: 0.5333 - val_loss: 1.8639 - val_accuracy: 0.4855
Epoch 4/20
832/832 [==============================] - 18s 22ms/step - loss: 1.4547 - accuracy: 0.5613 - val_loss: 1.8619 - val_accuracy: 0.4890
Epoch 5/20
832/832 [==============================] - 18s 22ms/step - loss: 1.5473 - accuracy: 0.5576 - val_loss: 2.3160 - val_accuracy: 0.3997
Epoch 6/20
832/832 [==============================] - 18s 22ms/step - loss: 1.7582 - accuracy: 0.4944 - val_loss: 1.9150 - val_accuracy: 0.4802
Epoch 7/20
832/832 [==============================] - 18s 22ms/step - loss: 1.3969 - accuracy: 0.5761 - val_loss: 1.9399 - val_accuracy:

KeyboardInterrupt: 